In [1]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리|
import random

## 노드의 블록 정보 확인

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## transaction 입력 및 채굴(기존과 동일)

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 100,
        "smart_contract": {}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드의 블록정보 확인 - 1

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## Smart Contract transaction 입력하기

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {"contract_code":"print('Ｈello Ｓmart－Ｃontract')"}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
json.loads(result)

In [ ]:
contract_address = json.loads(result)['contract_address']

## 노드의 블록정보 확인 - 2

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

##  채굴하기 - 1

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드의 블록정보 확인 - 3

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## Smart Contract 호출 -1

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

In [ ]:
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])            

## Smart Contract 입력하기 -2  / NFT

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {"contract_code":"""
myNFT = {'NFT_NAME':'SMARTCONTRACT_NFT',
         'NFT_URL': 'https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png'}
                                            """}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

# 채굴을 통하여 거래내역을 블록에 저장한다
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
contract_address

In [ ]:
# 블록 정보 호출
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

## 스마트 컨트랙트를 호출 및 실행
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])            
myNFT

## Smart Contract 입력하기 -3  / Token 만들기

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {
                           "contract_code" :"token_name = 'pyTOKEN' \ntoken_total_volume = 100000"
                                            }
            }
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
contract_address

In [ ]:
# 블록 정보 호출
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

## 스마트 컨트랙트를 호출 및 실행
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])       
            break
print(token_name , "/ ", token_total_volume  )

## Smart Contract 입력하기 -4  / 거래 가능한 Token 만들기

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {
                           "contract_code" :"token_name = 'pyTOKEN' \ntoken_total_volume = 100000\ntoken_owner = {'token_maker' : 10000}",
                           "contract_function_getBalance" :"""
def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]
""",
                           "contract_function_sendToken" :"""
def send_token(sender,recipent,amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender]  = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent]  = token_owner[recipent] + amount
            else :
                token_owner[recipent]  =  amount
            print("Transaction Completed")
            get_balance(sender) 
            get_balance(recipent) 

        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Sender id"
"""
                           

                                            }
            }
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']


headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])
            break

In [ ]:
token_name 

In [ ]:
token_total_volume

In [ ]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('token_maker')

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('token_maker','user',50)

In [ ]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('token_maker')

In [ ]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('user')

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('token_maker','user',50)

In [ ]:
send_token('token_maker','user2',3000)

In [ ]:
exec(_tx['smart_contract']['contract_function_getBalance'])
get_balance('user2')

## 참고 : 이더리움 네트워크 상에서 Layer2 토큰을 만드는 솔리디티 코드

In [ ]:
pragma solidity ^0.4.24;


contract SafeMath {
    function safeAdd(uint a, uint b) public pure returns (uint c) {
        c = a + b;
        require(c >= a);
    }
    function safeSub(uint a, uint b) public pure returns (uint c) {
        require(b <= a);
        c = a - b;
    }
    function safeMul(uint a, uint b) public pure returns (uint c) {
        c = a * b;
        require(a == 0 || c / a == b);
    }
    function safeDiv(uint a, uint b) public pure returns (uint c) {
        require(b > 0);
        c = a / b;
    }
}


/**
ERC Token Standard #20 Interface
https://github.com/ethereum/EIPs/blob/master/EIPS/eip-20-token-standard.md
*/

contract ERC20Interface {
    function totalSupply() public constant returns (uint);
    function balanceOf(address tokenOwner) public constant returns (uint balance);
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining);
    function transfer(address to, uint tokens) public returns (bool success);
    function approve(address spender, uint tokens) public returns (bool success);
    function transferFrom(address from, address to, uint tokens) public returns (bool success);

    event Transfer(address indexed from, address indexed to, uint tokens);
    event Approval(address indexed tokenOwner, address indexed spender, uint tokens);
}



contract ApproveAndCallFallBack {
    function receiveApproval(address from, uint256 tokens, address token, bytes data) public;
}

contract pyTOKEN is ERC20Interface, SafeMath {
    string public symbol;
    string public  name;
    uint8 public decimals;
    uint public _totalSupply;

    mapping(address => uint) balances;
    mapping(address => mapping(address => uint)) allowed;


    // ------------------------------------------------------------------------
    // 토큰의 명칭 및 총 발행량 선언
    // ------------------------------------------------------------------------
    constructor() public {
        symbol = "pyTOKEN";
        name = "Python Token";
        decimals = 2;
        _totalSupply = 100000;
        balances[0xAd61C057eD2cE6BB81f10eeC0192e3a7DEC848dc] = _totalSupply;
        emit Transfer(address(0), 0xAd61C057eD2cE6BB81f10eeC0192e3a7DEC848dc, _totalSupply);
    }


    // ------------------------------------------------------------------------
    // T총 발행량 설정
    // ------------------------------------------------------------------------
    function totalSupply() public constant returns (uint) {
        return _totalSupply  - balances[address(0)];
    }


    // ------------------------------------------------------------------------
    // 잔액 조회 함수
    // ------------------------------------------------------------------------
    function balanceOf(address tokenOwner) public constant returns (uint balance) {
        return balances[tokenOwner];
    }


    // ------------------------------------------------------------------------
    // 토큰 보내기 함수1
    // ------------------------------------------------------------------------
    function transfer(address to, uint tokens) public returns (bool success) {
        balances[msg.sender] = safeSub(balances[msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(msg.sender, to, tokens);
        return true;
    }


    // ------------------------------------------------------------------------
    // 잔고 조회후 토큰 보내기 기능 승인
    // ------------------------------------------------------------------------
    function approve(address spender, uint tokens) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        return true;
    }


    // ------------------------------------------------------------------------
    // 토큰 보내기 함수2
    // ------------------------------------------------------------------------
    function transferFrom(address from, address to, uint tokens) public returns (bool success) {
        balances[from] = safeSub(balances[from], tokens);
        allowed[from][msg.sender] = safeSub(allowed[from][msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(from, to, tokens);
        return true;
    }


    // ------------------------------------------------------------------------
    // 토큰을 보낼 수 있는지 잔고조회 확인
    // ------------------------------------------------------------------------
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining) {
        return allowed[tokenOwner][spender];
    }



}